In [1]:
import cv2
import math
import numpy as np
import sys
#Dark Channel Prior 
def DCP(im,size):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(size,size))
    dark = cv2.erode(dc,kernel)
    return dark

In [2]:
#window size( Please feel free to change the window size according to the actual size of the input image, which may increase the overall performance)
size=7
#input image( Please enter the route of the input image)
img_colour=cv2.imread('C:\\Users\\Gengqian Yang\\example1.jpg')
img_colour =img_colour.astype('float64')/255
img_dark=DCP(img_colour,size)
cv2.namedWindow("DCP",0)
cv2.resizeWindow("DCP", 1920, 1080)
cv2.imshow('DCP',img_dark)
cv2.waitKey()
cv2.destroyAllWindows()
num_pixel=np.size(img_dark)
print(np.shape(img_colour))
print(np.shape(img_dark))
print('The number of pixels is {}.'.format(num_pixel))

(450, 800, 3)
(450, 800)
The number of pixels is 360000.


In [3]:
#estimation function of the atmospheric light
def atmospheric_light(img_colour,img_dark):
    num_of_pixel=img_dark.size
    h,w=img_colour.shape[0:2]
    img_size = h*w
    num=int(num_of_pixel/1000)
    darkvec = img_dark.reshape(img_size,1)
    imvec = img_colour.reshape(img_size,3)
    indices = darkvec.argsort()
    indices = indices[img_size-num::]
    atmsum = np.zeros([1,3])
    for ind in range(1,num):
        atmsum = atmsum + imvec[indices[ind]]
    A = atmsum / num
    return A

In [4]:
#the top 0.1% brightest pixels
A=atmospheric_light(img_colour,img_dark)
#the brightest pixel
index=[np.where(img_dark == np.amax(img_dark))[0][0],np.where(img_dark == np.amax(img_dark))[1][0]]
#our proposed estimation method
A[0][0]=(float(img_colour[int(index[0])][int(index[1])][0]))*2/3+A[0][0]/3
A[0][1]=(float(img_colour[int(index[0])][int(index[1])][1]))*2/3+A[0][1]/3
A[0][2]=(float(img_colour[int(index[0])][int(index[1])][2]))*2/3+A[0][2]/3
print(A)
print(A*255)

[[0.71185548 0.69619826 0.69750182]]
[[181.52314815 177.53055556 177.86296296]]


In [5]:
#coarse transmission estimation function
def transmission(img_colour,A,sz):
    #There is no haze-free image in the real-world due to the wide existence of haze.
    #Hence, a coefficient omega was introduced to reserve a very small amount of haze in the result to produce a natural result.
    omega = 0.95
    img_normalization = np.empty(img_colour.shape,img_colour.dtype);
    for ind in range(0,3):
        img_normalization[:,:,ind] = img_colour[:,:,ind]/A[0,ind]
    transmission = 1 - omega*DCP(img_normalization,sz);
    return transmission

In [6]:
t=transmission(img_colour,A,size)
#checking coarse tranmission map by using opencv
cv2.namedWindow("transmission",0)
cv2.resizeWindow("transmission", 1920, 1080)
cv2.imshow('transmission',t)
cv2.waitKey()
cv2.destroyAllWindows()
print(t)
cv2.imwrite('t for example1.jpg',t*255)

[[0.22018616 0.22018616 0.22018616 ... 0.15217425 0.15217425 0.15217425]
 [0.22018616 0.22018616 0.22018616 ... 0.15217425 0.15217425 0.15217425]
 [0.22018616 0.22018616 0.22018616 ... 0.15217425 0.15217425 0.15217425]
 ...
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]]


True

In [7]:
# morphological operations
#closing
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
closing = cv2.morphologyEx(t, cv2.MORPH_CLOSE, kernel, iterations=2)
t_C=closing
#opening
opening = cv2.morphologyEx(t_C, cv2.MORPH_OPEN, kernel, iterations=2)
t_CO=opening

In [8]:
#checking tranmission CO by using opencv
cv2.namedWindow("transmission CO",0)
cv2.resizeWindow("transmission CO", 1920, 1080)
cv2.imshow('transmission CO',t_CO)
cv2.waitKey()
cv2.destroyAllWindows()
print(t_CO)
cv2.imwrite('t_CO for example1.jpg',t_CO*255)

[[0.22552735 0.22552735 0.22552735 ... 0.14694075 0.14694075 0.14694075]
 [0.22552735 0.22552735 0.22552735 ... 0.14694075 0.14694075 0.14694075]
 [0.22552735 0.22552735 0.22552735 ... 0.14694075 0.14694075 0.14694075]
 ...
 [0.69555213 0.69555213 0.69555213 ... 0.61009308 0.61009308 0.61009308]
 [0.69555213 0.69555213 0.69555213 ... 0.61009308 0.61009308 0.61009308]
 [0.69555213 0.69555213 0.69555213 ... 0.61009308 0.61009308 0.61009308]]


True

In [9]:
J = np.copy(img_colour)
#lower bound of the transmission map
t_main = cv2.max(t_CO,0.4)
t_R =t-t_CO
#reconstructed transmission map
t=t_main+t_R
print(t)
#dehazing by reverse calculation of the atmospheric scattering model
for ind in range(0,3):
    J[:,:,ind] = (img_colour[:,:,ind]-A[0,ind])/t + A[0,ind]

[[0.39465881 0.39465881 0.39465881 ... 0.40523349 0.40523349 0.40523349]
 [0.39465881 0.39465881 0.39465881 ... 0.40523349 0.40523349 0.40523349]
 [0.39465881 0.39465881 0.39465881 ... 0.40523349 0.40523349 0.40523349]
 ...
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]
 [0.71691689 0.71691689 0.71691689 ... 0.66350499 0.66350499 0.6581638 ]]


In [10]:
#checking haze-free image by using opencv
cv2.namedWindow("image",0)
cv2.resizeWindow("image", 1920, 1080)
cv2.imshow('image',img_colour)
cv2.namedWindow("haze_free image",0)
cv2.resizeWindow("haze_free image", 1920, 1080)
cv2.imshow('haze_free image',J)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
#save function
cv2.imwrite('C:\\Users\\Gengqian Yang\\haze_free image.jpg',J*255)

True

In [12]:
#CLAHE (This step is optional. It is only used for enhancing the contrast and luminance of a dim result, which is rare but possible.)
import cv2
path = 'C:\\Users\\Gengqian Yang\\haze_free image.jpg'
image = cv2.imread(path,cv2.IMREAD_COLOR)
cv2.imwrite('image.jpg',image,[cv2.IMWRITE_JPEG_QUALITY, 50])
b,g,r = cv2.split(image)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
b = clahe.apply(b)
g = clahe.apply(g)
r = clahe.apply(r)
image = cv2.merge([b,g,r])
cv2.imwrite('C:\\Users\\Gengqian Yang\\clahe.jpg',image,[cv2.IMWRITE_JPEG_QUALITY, 50])

True

In [13]:
#checking haze-free image after applying CLAHE by using opencv
cv2.namedWindow("CLAHE",0)
cv2.resizeWindow("CLAHE", 1920, 1080)
cv2.imshow('CLAHE',image)
cv2.waitKey()
cv2.destroyAllWindows()